<a href="https://colab.research.google.com/github/Mustafa4564/DS_LAB/blob/main/Experiment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uninformed Searching Techniques

### Breadth-First Search (BFS)

BFS is an algorithm for traversing or searching tree or graph data structures. It starts at the tree root (or some arbitrary node of a graph, sometimes referred to as a 'search key') and explores all of the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level.

In [1]:
from collections import deque

def bfs(graph, start):
    visited = set()
    queue = deque([start])
    visited.add(start)
    traversal_order = []

    while queue:
        vertex = queue.popleft()
        traversal_order.append(vertex)

        for neighbor in graph[vertex]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    return traversal_order


### Depth-First Search (DFS)

DFS is another algorithm for traversing or searching tree or graph data structures. The algorithm starts at the root node (selecting some arbitrary node as the root in the case of a graph) and explores as far as possible along each branch before backtracking.

In [2]:
def dfs(graph, start, visited=None, traversal_order=None):
    if visited is None:
        visited = set()
    if traversal_order is None:
        traversal_order = []

    visited.add(start)
    traversal_order.append(start)

    for neighbor in graph[start]:
        if neighbor not in visited:
            dfs(graph, neighbor, visited, traversal_order)
    return traversal_order


### Demonstrating BFS and DFS on a sample graph

Let's define a simple graph and see how both algorithms traverse it.

In [3]:
# Define a sample graph using an adjacency list
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

start_node = 'A'

print(f"Graph: {graph}")
print(f"Starting node for traversal: {start_node}")

print("\nBreadth-First Search (BFS) traversal:")
bfs_result = bfs(graph, start_node)
print(bfs_result)

print("\nDepth-First Search (DFS) traversal:")
dfs_result = dfs(graph, start_node)
print(dfs_result)


Graph: {'A': ['B', 'C'], 'B': ['A', 'D', 'E'], 'C': ['A', 'F'], 'D': ['B'], 'E': ['B', 'F'], 'F': ['C', 'E']}
Starting node for traversal: A

Breadth-First Search (BFS) traversal:
['A', 'B', 'C', 'D', 'E', 'F']

Depth-First Search (DFS) traversal:
['A', 'B', 'D', 'E', 'F', 'C']


# A* algorithm

## A* Search Algorithm

A* (pronounced "A-star") is a graph traversal and pathfinding algorithm, which is often used in computer science for tasks like finding the shortest path in a road network or a maze. It is a 'best-first' search algorithm, meaning that it explores the most promising paths first.

A* achieves its performance by using a heuristic function that estimates the cost from the current node to the goal node. It combines this estimate with the actual cost incurred so far from the start node to the current node.

**Key components of A*:**

*   **`g(n)`**: The cost from the start node to node `n`.
*   **`h(n)`**: The estimated cost (heuristic) from node `n` to the goal node.
*   **`f(n)`**: The total estimated cost of the path through node `n` to the goal, calculated as `f(n) = g(n) + h(n)`.

The algorithm maintains a set of discovered nodes to be evaluated (the 'open set' or 'priority queue') and a set of nodes already evaluated (the 'closed set'). It prioritizes exploring nodes with the lowest `f(n)` value. Once the goal node is reached, the path can be reconstructed by backtracking from the goal using the 'came_from' map.

In [4]:
import heapq

def a_star(graph, start, goal, heuristic):
    # The set of discovered nodes that may need to be (re-)expanded.
    # Initially, only the start node is known. This is a min-heap.
    open_set = [(0, start)]  # (f_score, node)

    # For node n, came_from[n] is the node immediately preceding it on the cheapest path from start to n currently known.
    came_from = {}

    # For node n, g_score[n] is the cost of the cheapest path from start to n currently known.
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0

    # For node n, f_score[n] = g_score[n] + h(n). f_score[n] represents our current best guess as to
    # how cheap a path from start to finish can be if it goes through n.
    f_score = {node: float('inf') for node in graph}
    f_score[start] = heuristic(start, goal)

    while open_set:
        # Get the node with the lowest f_score from the open_set
        current_f, current_node = heapq.heappop(open_set)

        if current_node == goal:
            # Reconstruct path if goal is reached
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start)
            return path[::-1], g_score[goal]

        # Explore neighbors
        for neighbor, weight in graph[current_node].items():
            # d(current_node, neighbor) is the weight of the edge from current_node to neighbor
            tentative_g_score = g_score[current_node] + weight

            if tentative_g_score < g_score[neighbor]:
                # This path to neighbor is better than any previous one. Record it!
                came_from[neighbor] = current_node
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None, None # No path found


### Sample Graph and Heuristic Function for A*

To demonstrate A*, we need a graph where nodes have positions and edges have weights. We'll use Manhattan distance as our heuristic function, which is suitable for grid-like movements (no diagonals).

In [5]:
# Define a sample graph with weighted edges (adjacency list with weights)
# Each node maps to a dictionary of its neighbors and the edge weights
graph_a_star = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'D': 2, 'E': 5},
    'C': {'A': 4, 'F': 2},
    'D': {'B': 2, 'G': 1},
    'E': {'B': 5, 'H': 3},
    'F': {'C': 2, 'I': 3},
    'G': {'D': 1, 'J': 4},
    'H': {'E': 3, 'J': 1},
    'I': {'F': 3, 'J': 1},
    'J': {'G': 4, 'H': 1, 'I': 1}
}

# Define coordinates for each node to calculate heuristic
coordinates = {
    'A': (0, 0),
    'B': (1, 0),
    'C': (0, 1),
    'D': (2, 0),
    'E': (1, 1),
    'F': (0, 2),
    'G': (3, 0),
    'H': (2, 1),
    'I': (1, 2),
    'J': (2, 2)
}

def manhattan_distance(node, goal):
    """Calculates Manhattan distance heuristic between two nodes."""
    x1, y1 = coordinates[node]
    x2, y2 = coordinates[goal]
    return abs(x1 - x2) + abs(y1 - y2)

# Set start and goal nodes
start_node_a_star = 'A'
goal_node_a_star = 'J'

print(f"A* Graph: {graph_a_star}")
print(f"Node Coordinates: {coordinates}")
print(f"Starting node for A*: {start_node_a_star}")
print(f"Goal node for A*: {goal_node_a_star}")

# Run A* algorithm
path, cost = a_star(graph_a_star, start_node_a_star, goal_node_a_star, manhattan_distance)

if path:
    print(f"\nShortest path from {start_node_a_star} to {goal_node_a_star}: {path}")
    print(f"Total cost: {cost}")
else:
    print(f"\nNo path found from {start_node_a_star} to {goal_node_a_star}")


A* Graph: {'A': {'B': 1, 'C': 4}, 'B': {'A': 1, 'D': 2, 'E': 5}, 'C': {'A': 4, 'F': 2}, 'D': {'B': 2, 'G': 1}, 'E': {'B': 5, 'H': 3}, 'F': {'C': 2, 'I': 3}, 'G': {'D': 1, 'J': 4}, 'H': {'E': 3, 'J': 1}, 'I': {'F': 3, 'J': 1}, 'J': {'G': 4, 'H': 1, 'I': 1}}
Node Coordinates: {'A': (0, 0), 'B': (1, 0), 'C': (0, 1), 'D': (2, 0), 'E': (1, 1), 'F': (0, 2), 'G': (3, 0), 'H': (2, 1), 'I': (1, 2), 'J': (2, 2)}
Starting node for A*: A
Goal node for A*: J

Shortest path from A to J: ['A', 'B', 'D', 'G', 'J']
Total cost: 8


# Minimax

## Minimax Algorithm

Minimax is a recursive algorithm for choosing the next move in an n-player game, usually a two-player game. A value is associated with each state or decision in the game. In two-player games, the Minimax algorithm is used to find the best move for a player, assuming that the opponent also plays optimally. It works by looking ahead at possible moves and their consequences, assigning a score to each final game state, and then working backward to determine the best move at the current state.

**Key concepts:**

*   **Maximizing Player**: Aims to get the highest possible score.
*   **Minimizing Player**: Aims to get the lowest possible score (which means maximizing the opponent's loss).
*   **Terminal State**: A game state where the game ends, and a utility value (score) is assigned.
*   **Recursion**: The algorithm explores the game tree recursively, alternating between maximizing and minimizing turns.

The algorithm evaluates the game tree by assigning a score to each node: if it's the maximizing player's turn, it chooses the maximum of its children's scores; if it's the minimizing player's turn, it chooses the minimum of its children's scores.

In [8]:
def minimax(node, depth, maximizing_player, game_tree):
    # Base case: if it's a direct score (integer) or depth limit is reached
    # and the node's 'children' are actual scores.
    if isinstance(node, int):
        return node

    # If the depth limit is reached, evaluate the scores directly from the node's children list.
    # This assumes that nodes like 'D', 'E', 'F', 'G' contain the final scores at depth 0.
    if depth == 0 and node in game_tree and all(isinstance(val, int) for val in game_tree[node]):
        if maximizing_player:
            return max(game_tree[node])
        else:
            return min(game_tree[node])

    # Recursive step
    if maximizing_player:
        max_eval = -float('inf')
        if node in game_tree:
            for child_node_or_value in game_tree[node]:
                evaluation = minimax(child_node_or_value, depth - 1, False, game_tree)
                max_eval = max(max_eval, evaluation)
        return max_eval
    else: # minimizing_player
        min_eval = float('inf')
        if node in game_tree:
            for child_node_or_value in game_tree[node]:
                evaluation = minimax(child_node_or_value, depth - 1, True, game_tree)
                min_eval = min(min_eval, evaluation)
        return min_eval


### Demonstrating Minimax on a sample game tree

Let's define a simple game tree where leaf nodes represent final scores for the maximizing player. We'll simulate a game with a certain depth.

In [9]:
# Define a simple game tree (represented as a dictionary)
# Keys are non-leaf nodes, values are lists of child nodes/scores
# Leaf nodes (scores) are integers
game_tree = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': [3, 5],  # Leaf nodes with scores
    'E': [2, 9],  # Leaf nodes with scores
    'F': [1, 0],  # Leaf nodes with scores
    'G': [8, 4]   # Leaf nodes with scores
}

start_node = 'A'
depth_limit = 2 # The depth of our example tree from root to leaf

print("Sample Game Tree:")
for parent, children in game_tree.items():
    print(f"  {parent} -> {children}")

print(f"\nStarting minimax from node '{start_node}' with depth limit {depth_limit}")

# Assuming the first player is the maximizing player
best_value = minimax(start_node, depth_limit, True, game_tree)

print(f"\nOptimal value for the maximizing player: {best_value}")


Sample Game Tree:
  A -> ['B', 'C']
  B -> ['D', 'E']
  C -> ['F', 'G']
  D -> [3, 5]
  E -> [2, 9]
  F -> [1, 0]
  G -> [8, 4]

Starting minimax from node 'A' with depth limit 2

Optimal value for the maximizing player: 5
